In [1]:
import pandas as pd
from pybedtools import BedTool
from refgene_parser import RefGene
import vcf

## Téléchargement des positions des Snps de la puce Omniexpress-24 
Les coordonnées correspondent au génome de référence hg19   
http://emea.support.illumina.com/downloads/infinium-omniexpress-24-v1-2-support-files.html?langsel=/fr/


In [5]:
!wget http://emea.support.illumina.com/content/dam/illumina-support/documents/downloads/productfiles/humanomniexpress-24/v1-2/infinium-omniexpress-24-v1-2-a1-bed.zip -O omniexpress.bed.zip
!unzip -o omniexpress.bed.zip
!mv InfiniumOmniExpress-24v1-2_A1.bed omniexpress.bed

--2019-03-22 12:43:48--  http://emea.support.illumina.com/content/dam/illumina-support/documents/downloads/productfiles/humanomniexpress-24/v1-2/infinium-omniexpress-24-v1-2-a1-bed.zip
Résolution de emea.support.illumina.com (emea.support.illumina.com)… 52.16.176.49, 52.31.118.253, 52.215.132.97
Connexion à emea.support.illumina.com (emea.support.illumina.com)|52.16.176.49|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 7511414 (7,2M) [application/zip]
Enregistre : «omniexpress.bed.zip»

omniexpress.bed.zip 100%[===================>]   7,16M  1,73MB/s    ds 4,3s    

2019-03-22 12:43:52 (1,66 MB/s) - «omniexpress.bed.zip» enregistré [7511414/7511414]

Archive:  omniexpress.bed.zip
  inflating: InfiniumOmniExpress-24v1-2_A1.bed  


## Téléchargement des noms des snps (rs) 
Certain SNPs sont nommé en VGxxx au lieu de rs.... Il y a un fichier qui fait le mapping de VG vers rs 

In [7]:
!wget http://emea.support.illumina.com/content/dam/illumina-support/documents/downloads/productfiles/humanomniexpress-24/v1-2/infinium-omniexpress-24-v1-2-a1-b144-rsids.zip -O rsids.zip
!unzip -o rsids.zip 
!mv InfiniumOmniExpress-24v1-2_A1_b144_rsids.txt rsids.txt
    

--2019-03-22 12:45:33--  http://emea.support.illumina.com/content/dam/illumina-support/documents/downloads/productfiles/humanomniexpress-24/v1-2/infinium-omniexpress-24-v1-2-a1-b144-rsids.zip
Résolution de emea.support.illumina.com (emea.support.illumina.com)… 52.31.118.253, 52.16.176.49, 52.215.132.97
Connexion à emea.support.illumina.com (emea.support.illumina.com)|52.31.118.253|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 3373501 (3,2M) [application/zip]
Enregistre : «rsids.zip»

rsids.zip           100%[===================>]   3,22M   772KB/s    ds 5,0s    

2019-03-22 12:45:39 (658 KB/s) - «rsids.zip» enregistré [3373501/3373501]

Archive:  rsids.zip
  inflating: InfiniumOmniExpress-24v1-2_A1_b144_rsids.txt  


## Télécargement de refgene hg19

In [18]:
!wget http://hgdownload.cse.ucsc.edu/goldenpath/hg19/database/refGene.txt.gz

--2019-03-22 12:50:34--  http://hgdownload.cse.ucsc.edu/goldenpath/hg19/database/refGene.txt.gz
Résolution de hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)… 128.114.119.163
Connexion à hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 3974990 (3,8M) [application/x-gzip]
Enregistre : «refGene.txt.gz»

refGene.txt.gz      100%[===================>]   3,79M   368KB/s    ds 13s     

2019-03-22 12:50:48 (295 KB/s) - «refGene.txt.gz» enregistré [3974990/3974990]



## Téléchargement de clinvar

In [59]:
!wget ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz
!gzip -d clinvar.vcf.gz

--2019-03-23 11:07:38--  ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh37/clinvar.vcf.gz
           => «clinvar.vcf.gz»
Résolution de ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)… 130.14.250.10, 2607:f220:41e:250::11
Connexion à ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:21… connecté.
Ouverture de session en tant que anonymous… Session établie.
==> SYST ... terminé.    ==> PWD ... terminé.
==> TYPE I ... terminé.  ==> CWD (1) /pub/clinvar/vcf_GRCh37 ... terminé.
==> SIZE clinvar.vcf.gz ... 18858717
==> PASV ... terminé.    ==> RETR clinvar.vcf.gz ... terminé.
Taille : 18858717 (18M) (non certifiée)

clinvar.vcf.gz      100%[===================>]  17,98M   915KB/s    ds 30s     

2019-03-23 11:08:10 (622 KB/s) - «clinvar.vcf.gz» enregistré [18858717]



## Transformation de clinvar.vcf en datafarame

In [8]:
data = []
vcf_reader = vcf.Reader(open('clinvar.vcf', 'r'))
for variant in vcf_reader:
    # On récupere que les snps 
    if variant.INFO["CLNVC"] == "single_nucleotide_variant":
        

        
        data.append((variant.CHROM,
                     variant.start,
                     variant.end, 
                     ",".join(variant.INFO.get("CLNSIG",[])),
                     ",".join(variant.INFO.get("CLNDN",[])),
                     "rs" + "".join(variant.INFO.get("RS",[])),
                      "".join(variant.INFO.get("CLNHGVS",[]))


                    ))
    
    
clinvar = pd.DataFrame(data)
clinvar.columns = ["chr","start","end","clinsig","clndn","rs","hgvs"]
clinvar.head()



,chr,start,end,clinsig,clndn,rs,hgvs
0,1,949421,949422,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...,rs143888043,NC_000001.10:g.949422G>A
1,1,949501,949502,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...,rs150861311,NC_000001.10:g.949502C>T
2,1,949522,949523,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...,rs786201005,NC_000001.10:g.949523C>T
3,1,949558,949559,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...,rs1553169766,NC_000001.10:g.949559C>T
4,1,949596,949597,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...,rs61766284,NC_000001.10:g.949597C>T


## Intersection entre clinvar et omniExpress via le rsid

In [9]:
omni_rs = pd.read_csv("rsids.txt",sep="\t")["RsID"]
omni_clinvar = clinvar[clinvar.rs.isin(omni_rs)]

## Garder que les snps pathogene

In [36]:
omni_clinvar_pathogene = omni_clinvar[omni_clinvar.clinsig == "Pathogenic"]
print("total pathogene", len(omni_clinvar_pathogene.rs.unique()))
omni_clinvar_pathogene.to_csv("pathogene.csv")

total pathogene 100


In [39]:
omni_clinvar_pathogene[["clndn","rs"]].to_csv("pathogene_clean.csv")

## Maladies associées 

In [66]:
# Export pour le blog 
import json
data = []
for i in omni_clinvar_pathogene[["clndn","rs"]].iterrows():
    data.append([i[1]["clndn"],i[1]["rs"]])

with open("/home/schutz/Dev/blog/output/test.json", "w") as file:
    file.write(json.dumps({"data":data}))



In [35]:
omni_clinvar_pathogene.clndn.unique()

array(['Familial_hypercholesterolemia',
       'Medium-chain_acyl-coenzyme_A_dehydrogenase_deficiency',
       'Medium-chain_acyl-coenzyme_A_dehydrogenase_deficiency|not_provided',
       'Congenital_sensory_neuropathy_with_selective_loss_of_small_myelinated_fibers',
       'Hemochromatosis_type_2A|Hemochromatosis_type_1',
       "Gaucher's_disease,_type_1", 'Prostate_cancer,_hereditary,_12',
       'not_provided', 'Lucey-Driscoll_syndrome',
       'Distal_myopathy,_Tateyama_type', 'Achondroplasia',
       'Ciliary_dyskinesia,_primary,_3',
       'Sandhoff_disease,_infantile_type',
       'Bifunctional_peroxisomal_enzyme_deficiency',
       'Cancer_progression_and_tumor_cell_motility',
       'Factor_XIII_subunit_A_deficiency',
       'Inborn_genetic_diseases|Classic_congenital_adrenal_hyperplasia_due_to_21-hydroxylase_deficiency|not_provided',
       'Glycogen_storage_disease_type_X|not_provided', 'Cystic_fibrosis',
       'Cystic_fibrosis|not_provided',
       'Cystic_fibrosis|Heredi